# Необходимые библиотеки

Для загрузки моделей

In [2]:
!pip install datasets transformers[sentencepiece]

     |████████████████████████████████| 325 kB 4.9 MB/s 
     |████████████████████████████████| 4.0 MB 56.5 MB/s 
     |████████████████████████████████| 1.1 MB 48.5 MB/s 
     |████████████████████████████████| 212 kB 58.0 MB/s 
     |████████████████████████████████| 77 kB 4.9 MB/s 
     |████████████████████████████████| 136 kB 47.6 MB/s 
     |████████████████████████████████| 127 kB 58.0 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 144 kB 58.9 MB/s 
     |████████████████████████████████| 271 kB 58.6 MB/s 
     |████████████████████████████████| 596 kB 48.7 MB/s 
     |████████████████████████████████| 6.6 MB 8.4 MB/s 
     |████████████████████████████████| 895 kB 56.2 MB/s 
     |████████████████████████████████| 1.2 MB 52.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    

In [3]:
from transformers import AutoTokenizer, AutoModel
import torch
import pickle
from tqdm import tqdm
from scipy.spatial.distance import cdist
from sklearn.metrics.pairwise import cosine_similarity

# Используем `sbert_large_nlu`

In [4]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_nlu_ru")
model = AutoModel.from_pretrained("sberbank-ai/sbert_large_nlu_ru")
dictionary = tokenizer.vocab
print("Фрагмент словаря модели:")
print(list(dictionary.keys())[:20])

Downloading:   0%|          | 0.00/323 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/655 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.70M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.59G [00:00<?, ?B/s]

Фрагмент словаря модели:
['Деви', '[unused84]', 'изви', 'инфраструктуры', 'американском', 'барак', 'незамыслова', 'прозвищу', '##гвардей', '##цере', 'своевременно', 'Григорий', 'застыл', '##вочных', '##ross', 'неотрази', 'туфли', '##ПЧ', '##oda', 'мессы']


Для более корректной работы

In [5]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

Пример перевода в векторное представление фрагмента словаря

In [6]:
#Sentences we want sentence embeddings for
sentences = list(dictionary.keys())[:20]

#Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=24, return_tensors='pt')

#Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

#Perform pooling. In this case, mean pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

sentence_embeddings

tensor([[ 2.2477e-01, -1.3175e-02, -4.4907e-01,  ..., -3.1239e-01,
         -3.2641e-01,  4.1755e-01],
        [ 4.9440e-01, -3.7576e-01, -3.2482e-01,  ...,  1.0678e-01,
         -6.0915e-01,  2.2228e-04],
        [ 3.2402e-01, -8.2163e-02, -4.0684e-01,  ...,  6.0229e-02,
         -2.1402e-01, -1.3248e-01],
        ...,
        [ 6.2086e-02, -5.4661e-02, -2.1828e-01,  ...,  1.1114e-01,
         -5.4950e-01, -1.6467e-01],
        [ 2.0413e-01,  1.0347e-01, -4.1801e-01,  ...,  1.3478e-01,
         -5.5464e-01, -9.5645e-02],
        [ 3.6988e-01, -8.8180e-02, -5.3876e-01,  ..., -1.1174e+00,
         -3.1749e-01, -1.6571e-01]])

# Перевод словаря модели в векторные представления (эмбеддинги-тензоры)

In [ ]:
dict_embeddings = {}

dictionary = list(dictionary.keys())

for word in tqdm(dictionary):
  encoded_input = tokenizer(word, padding=True, truncation=True, max_length=24, return_tensors='pt')
  with torch.no_grad():
    model_output = model(**encoded_input)
  embedding = mean_pooling(model_output, encoded_input['attention_mask'])
  dict_embeddings[word] = embedding

# Запись полученных эмбеддингов в файл, чтобы не делать этого при каждом запуске

with open('dict_embeddings.pickle', 'wb') as f:
  pickle.dump(dict_embeddings, f)

Загрузка файла

In [ ]:
with open('dict_embeddings.pickle', 'rb') as f:
  dict_embeddings = pickle.load(f)

print("Количество полученных эмбеддингов", len(dict_embeddings))

120138

# Поиск взаимозаменяемых слов по сходству векторов

Введите в `word` своё слово

In [ ]:
word = "университет"

Эмбеддинг для ```text```

In [ ]:
encoded_input = tokenizer(word, padding=True, truncation=True, max_length=24, return_tensors='pt')
with torch.no_grad():
    model_output = model(**encoded_input)
word_embedding = mean_pooling(model_output, encoded_input['attention_mask'])

print(f"Эмбеддинг: {word_embedding}")

Эмбеддинг: tensor([ 0.4884, -0.4788,  0.6102,  ..., -0.1668,  0.3430, -0.3652])


Расчет схожести слов

- по евклидову расстоянию ```euclidean_dist```
- по косиноснуму сходству ```cosine_sim```

In [ ]:
from scipy.spatial.distance import cdist
from sklearn.metrics.pairwise import cosine_similarity

euclidean_dist = {}
cosine_sim = {}

for word in tqdm(dict_embeddings):

      word_embedding = dict_embeddings[word]

      euclidean_dist[word] = 100 - (cdist(word_embedding.reshape(1,-1), 
                                   word_embedding.reshape(1,-1), 
                                   metric='euclidean')[0][0])
      cosine_sim[word] = (cosine_similarity(word_embedding.reshape(1,-1), 
                                           word_embedding.reshape(1,-1))[0][0]) * 100

100%|██████████| 120138/120138 [00:47<00:00, 2524.32it/s]


Топ слов по схожести

- Евклидово расстояние:

In [ ]:
sorted_tuples = sorted(euclidean_dist.items(), key=lambda item: item[1])
euclidean_dist = {k: v for k, v in sorted_tuples}
for i, j in zip(list(euclidean_dist.keys())[-10:], list(euclidean_dist.values())[-10:]):
  print(i, j)

бытовом 87.43234806718017
школьниц 87.585745469591
здание 87.61813242156639
газопровод 87.62179652156027
либералов 87.82348287887778
трон 88.01494920194176
прогни 88.17724603305031
дорога 88.19715160897564
корп 88.68821357950794
университет 100.0


- Косинус сходства:

In [ ]:
sorted_tuples = sorted(cosine_sim.items(), key=lambda item: item[1])
cosine_sim = {k: v for k, v in sorted_tuples}
for i, j in zip(list(cosine_sim.keys())[-10:], list(cosine_sim.values())[-10:]):
  print(i, j)

бытовом 80.58443069458008
школьниц 80.90823292732239
здание 81.08738660812378
газопровод 81.15297555923462
либералов 81.79396390914917
трон 82.32142329216003
дорога 82.6618492603302
прогни 82.73684978485107
корп 84.23560857772827
университет 100.0
